In [12]:
# Import dependencies 
import plotly.express as px
import pandas as pd
import numpy as np
import random
from scipy import constants
from sklearn import linear_model
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_regression
from sklearn.datasets import make_classification
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import LogisticRegression

from scipy.spatial.distance import pdist, squareform
from scipy.stats import pearsonr

import lightgbm as lgb

%matplotlib nbagg
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

from matplotlib.patches import Polygon

import pytz
from IPython.display import HTML
from matplotlib import animation, rc
from matplotlib.patches import Rectangle, Arrow, FancyArrow
from matplotlib.patches import Polygon
import matplotlib.patheffects as pe
import gc


import matplotlib.patches as patches
import matplotlib._color_data as mcd
import matplotlib.patches as mpatch
from scipy.spatial import ConvexHull

from tqdm import tqdm
from datetime import date
from datetime import datetime
import io
import time
import io
import re

from shapely.geometry import Point, Polygon, GeometryCollection,MultiPoint
from shapely.validation import make_valid


pd.options.mode.chained_assignment = None 
pd.set_option('display.max_columns', None)
import math

import warnings 
warnings.filterwarnings("ignore")

import seaborn as sns





In [2]:
pd.options.mode.chained_assignment = None 
pd.set_option('display.max_columns', None)

import warnings 
warnings.filterwarnings("ignore")

from raceplotly.plots import barplot

# Import Data
games = pd.read_csv("E:\\coding\\NFL_Big_Data_Bowl_2023\\NFL_Performance_Linemen\\resources\\csv_files\\2023\\games.csv")
plays = pd.read_csv("E:\\coding\\NFL_Big_Data_Bowl_2023\\NFL_Performance_Linemen\\resources\\csv_files\\2023\\plays.csv")
players = pd.read_csv("E:\\coding\\NFL_Big_Data_Bowl_2023\\NFL_Performance_Linemen\\resources\\csv_files\\2023\\players.csv")
week1 = pd.read_csv("E:\\coding\\NFL_Big_Data_Bowl_2023\\NFL_Performance_Linemen\\resources\\csv_files\\2023\\week1.csv", low_memory=False)
week2 = pd.read_csv("E:\\coding\\NFL_Big_Data_Bowl_2023\\NFL_Performance_Linemen\\resources\\csv_files\\2023\\week2.csv", low_memory=False)
week3 = pd.read_csv("E:\\coding\\NFL_Big_Data_Bowl_2023\\NFL_Performance_Linemen\\resources\\csv_files\\2023\\week3.csv", low_memory=False)
week4 = pd.read_csv("E:\\coding\\NFL_Big_Data_Bowl_2023\\NFL_Performance_Linemen\\resources\\csv_files\\2023\\week4.csv", low_memory=False)
week5 = pd.read_csv("E:\\coding\\NFL_Big_Data_Bowl_2023\\NFL_Performance_Linemen\\resources\\csv_files\\2023\\week5.csv", low_memory=False)
week6 = pd.read_csv("E:\\coding\\NFL_Big_Data_Bowl_2023\\NFL_Performance_Linemen\\resources\\csv_files\\2023\\week6.csv", low_memory=False)
week7 = pd.read_csv("E:\\coding\\NFL_Big_Data_Bowl_2023\\NFL_Performance_Linemen\\resources\\csv_files\\2023\\week7.csv", low_memory=False)
week8 = pd.read_csv("E:\\coding\\NFL_Big_Data_Bowl_2023\\NFL_Performance_Linemen\\resources\\csv_files\\2023\\week8.csv", low_memory=False)

pffScoutingData = pd.read_csv("E:\\coding\\NFL_Big_Data_Bowl_2023\\NFL_Performance_Linemen\\resources\\csv_files\\2023\\pffScoutingData.csv")

tracking = week1.append([
    week2,
    week3,
    week4,
    week5,
    week6,
    week7,
    week8,])

# Merge Data

joined_all = pd.merge(games,plays,how="inner",on = "gameId")
joined_all = pd.merge(joined_all,tracking,how="inner",on=["gameId","playId"])
# left join on players to keep football records
joined_all = pd.merge(joined_all,players,how="left",on = "nflId")
joined_all = pd.merge(joined_all,pffScoutingData,how="left",on=["gameId","playId",'nflId'])


In [3]:
games.columns

Index(['gameId', 'season', 'week', 'gameDate', 'gameTimeEastern',
       'homeTeamAbbr', 'visitorTeamAbbr'],
      dtype='object')

In [4]:
plays.columns

Index(['gameId', 'playId', 'playDescription', 'quarter', 'down', 'yardsToGo',
       'possessionTeam', 'defensiveTeam', 'yardlineSide', 'yardlineNumber',
       'gameClock', 'preSnapHomeScore', 'preSnapVisitorScore', 'passResult',
       'penaltyYards', 'prePenaltyPlayResult', 'playResult', 'foulName1',
       'foulNFLId1', 'foulName2', 'foulNFLId2', 'foulName3', 'foulNFLId3',
       'absoluteYardlineNumber', 'offenseFormation', 'personnelO',
       'defendersInBox', 'personnelD', 'dropBackType', 'pff_playAction',
       'pff_passCoverage', 'pff_passCoverageType'],
      dtype='object')

In [5]:
players.columns

Index(['nflId', 'height', 'weight', 'birthDate', 'collegeName',
       'officialPosition', 'displayName'],
      dtype='object')

In [6]:
week1.columns

Index(['gameId', 'playId', 'nflId', 'frameId', 'time', 'jerseyNumber', 'team',
       'playDirection', 'x', 'y', 's', 'a', 'dis', 'o', 'dir', 'event'],
      dtype='object')

In [7]:
pffScoutingData.columns

Index(['gameId', 'playId', 'nflId', 'pff_role', 'pff_positionLinedUp',
       'pff_hit', 'pff_hurry', 'pff_sack', 'pff_beatenByDefender',
       'pff_hitAllowed', 'pff_hurryAllowed', 'pff_sackAllowed',
       'pff_nflIdBlockedPlayer', 'pff_blockType', 'pff_backFieldBlock'],
      dtype='object')

In [8]:
# Merge Data

joined_all = pd.merge(games,plays,how="inner",on = "gameId")
joined_all = pd.merge(joined_all,week1,how="inner",on=["gameId","playId"])
# left join on players to keep football records
joined_all = pd.merge(joined_all,players,how="left",on = "nflId")
joined_all = pd.merge(joined_all,pffScoutingData,how="left",on=["gameId","playId",'nflId'])

# Below is needed to pick a random play when displaying a data frame
# play_focus = random.choice(joined_all['playId'].tolist())
# focused_df = joined_all[(joined_all.playId==play_focus)]

In [10]:
joined_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1118122 entries, 0 to 1118121
Data columns (total 70 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   gameId                  1118122 non-null  int64  
 1   season                  1118122 non-null  int64  
 2   week                    1118122 non-null  int64  
 3   gameDate                1118122 non-null  object 
 4   gameTimeEastern         1118122 non-null  object 
 5   homeTeamAbbr            1118122 non-null  object 
 6   visitorTeamAbbr         1118122 non-null  object 
 7   playId                  1118122 non-null  int64  
 8   playDescription         1118122 non-null  object 
 9   quarter                 1118122 non-null  int64  
 10  down                    1118122 non-null  int64  
 11  yardsToGo               1118122 non-null  int64  
 12  possessionTeam          1118122 non-null  object 
 13  defensiveTeam           1118122 non-null  object 
 14  ya

In [16]:
# create an empty output dataframe to append data into 
df_presnap = pd.DataFrame()

# loop through each game 
for gid in joined_all['gameId'].unique():
  # subset the data down to one game
  one_game = joined_all.loc[joined_all['gameId']==gid]

  # loop through each play
  for pid in one_game['playId'].unique():

    # subset game data down to one play 
    one_play = one_game.loc[one_game['playId']==pid]

    # isolate the first frame of data only (you could also loop through all frames if you so choose. this example is supposed to be a short demo)
    one_frame = one_play.loc[one_play['frameId']==1]

    # reset index (saving the original index as a column) and set new index as player ID 
    one_frame = one_frame.reset_index().set_index('nflId')

    # turn PFF player positioning info into unique positions given the play (for modeling purposes)
    one_frame['pff_positionLinedUp'] = one_frame['pff_positionLinedUp'] + one_frame.groupby('pff_positionLinedUp', as_index=False).cumcount().astype(str).str.replace('0', '')
    
    # get pairwise distance, turn it into a pairwise matrix, set it in a pandas dataframe with index as nflId and columns as positions 
    _df = pd.DataFrame(squareform(pdist(one_frame.loc[:, ['x','y']])), index=one_frame.index, columns=one_frame['pff_positionLinedUp'].unique())

    # concat pairwise matrix column-wise onto original one_frame data
    one_frame = pd.concat([one_frame,_df], axis=1).rename(columns={np.nan:'dist_from_ball'})

    # change index back to nflId column, set the column "index" to the true index 
    one_frame = one_frame.reset_index().set_index('index')

    # clear the index name for prettyfication 
    one_frame.index.name = None

    # append to output dataframe
    df_presnap = df_presnap.append(one_frame)

# fill the nulled sparse positions with -1, indicating that position was not apparent on a given play and/or frame
positions = [x for x in df_presnap.columns.values if x not in one_game.columns.values]
df_presnap.loc[:, positions] = df_presnap.loc[:, positions].fillna(-1)    

In [21]:
# Random samples of distances away from the football and QB 
df_presnap.loc[:, ['nflId', 'pff_positionLinedUp', 'gameId', 'playId','QB', 'dist_from_ball']].sample(5)

,nflId,pff_positionLinedUp,gameId,playId,QB,dist_from_ball
979175,48159.0,C,2021091213,684,4.290571,0.282843
728791,41264.0,LG,2021091209,126,1.856906,2.190457
844902,52519.0,LG,2021091210,3415,4.124197,1.644019
48647,46132.0,RCB,2021090900,2554,11.037341,9.644361
467552,41246.0,SLWR,2021091205,513,9.501705,9.050182


In [25]:
# let's define "pressure allowed" as any time a player allowed a hit, sack, or hurry as defined by PFF's scouting data
df_presnap['pressure_allowed'] = df_presnap.loc[:, ['pff_hitAllowed', 'pff_hurryAllowed', 'pff_sackAllowed']].sum(axis=1)

# subset the data to only players on the "pass block" 
model_data = df_presnap.loc[df_presnap['pff_role']=='Pass Block']

# let's use an extremely simple feature set: a player's XY position and all the distances away from each player + the ball
feats = ['x', 'y'] + positions

# try to predict our new feature "pressure_allowed"
target = 'pressure_allowed'

# 5 fold group K-fold (where we group games together so as to not overfit on any potential game result leakage)
kf = GroupKFold(5)

# make an empty series to contain the new preds
preds = pd.Series()

# loop over the dataset, 80% train data, 20% test data
for train_idx, test_idx in kf.split(model_data, groups=model_data['gameId']):

  # split your train and test data by integer-position (aka 0 = row 0, 1 = row 1, etc)
  # if you aren't using sklearn pipeline or other convenience sklearn functions
  # KFold splits are outputted as integer-positions, not label-positions. so you will need to use .iloc (not .loc)
  train_data = model_data.iloc[train_idx]
  test_data = model_data.iloc[test_idx]
  
  # train a simple logistic regression, default parameters blindly accepted
  model = LogisticRegression()
  model.fit(train_data.loc[:, feats], train_data[target])

  # predict on test set and store predictions to a series
  # remember since we're potentially going from numpy outputs to pandas, we can explicitly set our index to the test_data index for assurance in the concatenation
  _s = pd.DataFrame(model.predict_proba(test_data.loc[:, feats]), index=test_data.index)[1]
  
  # append preds to the preds series output
  preds = preds.append(_s)

# concatenate predictions to model_data dataset
model_data[f"{target}_pred"] = preds

model_data

,nflId,gameId,season,week,gameDate,gameTimeEastern,homeTeamAbbr,visitorTeamAbbr,playId,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,gameClock,preSnapHomeScore,preSnapVisitorScore,passResult,penaltyYards,prePenaltyPlayResult,playResult,foulName1,foulNFLId1,foulName2,foulNFLId2,foulName3,foulNFLId3,absoluteYardlineNumber,offenseFormation,personnelO,defendersInBox,personnelD,dropBackType,pff_playAction,pff_passCoverage,pff_passCoverageType,frameId,time,jerseyNumber,team,playDirection,x,y,s,a,dis,o,dir,event,height,weight,birthDate,collegeName,officialPosition,displayName,pff_role,pff_positionLinedUp,pff_hit,pff_hurry,pff_sack,pff_beatenByDefender,pff_hitAllowed,pff_hurryAllowed,pff_sackAllowed,pff_nflIdBlockedPlayer,pff_blockType,pff_backFieldBlock,QB,TE-L,LWR,HB-R,C,RWR,LEO,LT,ROLB,LG,ROLB1,RCB,SLWR,SCBR,DRT,FS,RG,LILB,RT,LCB,LILB1,RE,dist_from_ball,RLB,SCBoL,SRoWR,DLT,SRiWR,SCBiL,REO,RILB,SCBiL1,SSR,LLB,LE,LOLB,NLT,TE-oR,LLB1,TE-iR,HB,FSR,MLB,HB-L,TE-R,SRWR,SCBL,NT,FSL,SSL,NRT,SS,SLoWR,SLiWR,SCBiR,SCBoR,LEO1,TE-oL,TE-iL,LOLB1,DRT1,SCBL1,LCB1,SCBR1,SCBoL1,SCBiR1,FS1,FB-L,FB,LLB2,RLB1,RILB1,MLB1,ROLB2,SCBoR1,RCB1,FSL1,FB-R,FSR1,LOLB2,REO1,DLT1,FSR2,SS1,pressure_allowed,pressure_allowed_pred
172,40151.0,2021090900,2021,1,09/09/2021,20:20:00,TB,DAL,97,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,13:33,0,0,I,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,43.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0,Cover-1,Man,1,2021-09-10T00:26:31.100,66.0,TB,right,42.18,23.97,0.0,0.0,0.01,45.36,206.92,None,6-4,319.0,1991-05-27,Colorado State-Pueblo,C,Ryan Jensen,Pass Block,C,NaN,NaN,NaN,0.0,0.0,0.0,0.0,44955.0,SW,0.0,4.417080,5.295508,12.825327,4.945068,0.000000,7.781857,5.473874,2.987189,8.914084,1.705579,6.142524,13.643039,12.920163,14.003517,1.741551,16.760251,1.540519,6.289777,2.805887,7.812324,2.590174,3.093558,0.743303,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,0.0,0.041017
301,42377.0,2021090900,2021,1,09/09/2021,20:20:00,TB,DAL,97,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,13:33,0,0,I,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,43.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0,Cover-1,Man,1,2021-09-10T00:26:31.100,76.0,TB,right,41.55,26.89,0.0,0.0,0.00,148.02,132.28,None,6-6,338.0,1993-06-23,Penn State,T,Donovan Smith,Pass Block,LT,NaN,NaN,NaN,0.0,0.0,1.0,0.0,42403.0,PP,0.0,4.627883,2.308354,9.865906,6.125112,2.987189,10.709883,8.457435,0.000000,6.307963,1.293406,4.085890,10.835091,9.937832,11.282491,2.597865,17.055676,4.261983,7.943815,5.544448,10.797541,5.455795,2.151279,3.162183,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,1.0,0.077511
387,42404.0,2021090900,2021,1,09/09/2021,20:20:00,TB,DAL,97,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,13:33,0,0,I,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,43.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0,Cover-1,Man,1,2021-09-10T00:26:31.100,74.0,TB,right,41.95,25.66,0.0,0.0,0.00,108.14,151.12,None,6-4,307.0,1993-04-17,Hobart,G,Ali Marpet,Pass Block,LG,NaN,NaN,NaN,0.0,0.0,0.0,0.0,44955.0,PP,0.0,4.421086,3.596804,11.121511,5.568492,1.705579,9.470190,7.166987,1.293406,7.331255,0.000000,4.759328,11.989316,11.215311,12.385350,1.606487,16.733541,3.076004,7.019095,4.373786,9.517069,4.169652,2.011890,1.888200,-1.0,-1.0,-1.0,-1.0,-

In [23]:
# groupby sum the target and the predicted target
df_plt = (model_data
          .loc[:, ['nflId', target, f'{target}_pred']]
          .groupby('nflId', as_index=False)
          .sum())

# calculate target over expected (pressures above or below prediction -- positive means worse pass blocking, negative means better pass blocking)
df_plt[f'{target}_over_expected'] = df_plt[target].sub(df_plt[f"{target}_pred"])

# join the names of the nfl players onto the dataset
df_plt = players.loc[:, ['nflId', 'displayName']].merge(df_plt)

# calculate R2 to show explained variance
r2 = pearsonr(df_plt[target], df_plt[f"{target}_pred"])[0]**2

# plot data
sns.regplot(data=df_plt, x=target, y=f'{target}_pred')
plt.title(f"model with distance features\nR2: {round(r2, 3)}")
sns.despine()
plt.show()

<IPython.core.display.Javascript object>

In [26]:
# This is sorted by actual pressures over the week 
df_plt.sort_values(target, ascending=False).head(10)

,nflId,displayName,pressure_allowed,pressure_allowed_pred,pressure_allowed_over_expected
266,52483,Jonah Jackson,10.0,5.359550,4.640450
40,41222,Alejandro Villanueva,10.0,3.152280,6.847720
98,43295,Ronnie Stanley,9.0,2.598281,6.401719
304,53497,Jalen Mayfield,8.0,2.798167,5.201833
279,52554,Tyler Biadasz,8.0,1.974140,6.025860
78,42410,A.J. Cann,8.0,3.200036,4.799964
137,44875,Dion Dawkins,7.0,3.424571,3.575429
84,42471,Jon Feliciano,7.0,3.602088,3.397912
249,48455,Matt Nelson,7.0,6.842104,0.157896
295,53443,Alijah Vera-Tucker,6.0,2.180735,3.819265


In [28]:
# This is sorted by predicted pressures over the week 
df_plt.sort_values(f'{target}_pred', ascending=False).head(10)

,nflId,displayName,pressure_allowed,pressure_allowed_pred,pressure_allowed_over_expected
249,48455,Matt Nelson,7.0,6.842104,0.157896
293,53436,Penei Sewell,6.0,6.356387,-0.356387
296,53446,Alex Leatherwood,5.0,5.559201,-0.559201
266,52483,Jonah Jackson,10.0,5.359550,4.640450
116,43453,Halapoulivaati Vaitai,5.0,5.141873,-0.141873
102,43320,Germain Ifedi,1.0,5.091865,-4.091865
181,46131,Brian O'Neill,0.0,5.050825,-5.050825
178,46106,Braden Smith,4.0,4.863896,-0.863896
273,52517,John Simpson,4.0,4.827836,-0.827836
82,42445,Daryl Williams,6.0,4.668833,1.331167


In [29]:
# These are the players who allowed the most pressures over expected (poor performances in this week)
df_plt.sort_values(f'{target}_over_expected', ascending=False).head(10)

,nflId,displayName,pressure_allowed,pressure_allowed_pred,pressure_allowed_over_expected
40,41222,Alejandro Villanueva,10.0,3.152280,6.847720
98,43295,Ronnie Stanley,9.0,2.598281,6.401719
279,52554,Tyler Biadasz,8.0,1.974140,6.025860
304,53497,Jalen Mayfield,8.0,2.798167,5.201833
78,42410,A.J. Cann,8.0,3.200036,4.799964
266,52483,Jonah Jackson,10.0,5.359550,4.640450
295,53443,Alijah Vera-Tucker,6.0,2.180735,3.819265
137,44875,Dion Dawkins,7.0,3.424571,3.575429
140,44882,Pat Elflein,5.0,1.455489,3.544511
84,42471,Jon Feliciano,7.0,3.602088,3.397912


In [30]:
# These are the players who allowed the least pressures over expected (good performances in this week)
df_plt.sort_values(f'{target}_over_expected', ascending=False).tail(10)

,nflId,displayName,pressure_allowed,pressure_allowed_pred,pressure_allowed_over_expected
167,46075,Quenton Nelson,1.0,3.668155,-2.668155
17,38553,Riley Reiff,0.0,2.833631,-2.833631
33,40089,Oday Aboushi,0.0,2.846759,-2.846759
148,44972,Jermaine Eluemunor,1.0,4.055714,-3.055714
9,37082,Tyron Smith,1.0,4.225206,-3.225206
294,53442,Rashawn Slater,0.0,3.346264,-3.346264
41,41232,Jake Matthews,0.0,3.673490,-3.673490
102,43320,Germain Ifedi,1.0,5.091865,-4.091865
62,41939,Andrew Norwell,0.0,4.665152,-4.665152
181,46131,Brian O'Neill,0.0,5.050825,-5.050825
